# CT-AGI

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/fuyu-quant/CreateTool-AGI/blob/main/examples/ctagi.ipynb)

In [1]:
!pip install git+https://github.com/fuyu-quant/CreateTool-AGI.git

  Cloning https://github.com/fuyu-quant/CreateTool-AGI.git to /tmp/pip-req-build-pz57wo1g
  Running command git clone --filter=blob:none --quiet https://github.com/fuyu-quant/CreateTool-AGI.git /tmp/pip-req-build-pz57wo1g
  Resolved https://github.com/fuyu-quant/CreateTool-AGI.git to commit 25937a63f3f08e5f4f9734575e179dbbe309093c
  Preparing metadata (setup.py) ... done
  Created wheel for createtoolagi: filename=createtoolagi-0.1.2-py3-none-any.whl size=8366 sha256=02cbce7bd96504500920763cb5623f749129e9114d10ccaa01e6ad8b44db0cf2
  Stored in directory: /tmp/pip-ephem-wheel-cache-n7f9brss/wheels/7c/ba/46/b1c3a7d693e2bf0b074d226b40ad5cc8f9d360978e0e7fe5d3
Successfully built createtoolagi
  Attempting uninstall: createtoolagi
    Found existing installation: createtoolagi 0.1.1
    Uninstalling createtoolagi-0.1.1:
      Successfully uninstalled createtoolagi-0.1.1


In [2]:
%%capture
!pip install langchain==0.0.167
!pip install openai==0.27.4
!pip install tiktoken==0.4.0
!pip install qdrant-client==1.2.0

In [2]:
from createtoolagi import CTAGI

ModuleNotFoundError: No module named 'creator'